# Advanced output handling


For grasping the basic idea of output handlers, check out this introduction: [validating agent output](validating_agent_output.html).

The purpose of this example is to demonstrate two concepts:

- Obtaining structured output from the agent
- Validating the output against the prompt that was given to the agent

This should give a good understanding of output handler semantics and prepare you to build your own ones.

In [3]:
%pip install python-dotenv
%pip install motleycrew

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
from dotenv import load_dotenv
load_dotenv()

from motleycrew import MotleyCrew
from motleycrew.agents.langchain import ReActToolCallingMotleyAgent
from motleycrew.tasks import SimpleTask
from motleycrew.common.exceptions import InvalidOutput

from motleycrew.tools import PythonREPLTool
from motleycrew.tools import LLMTool
from motleycrew.tools import MotleyTool

2024-10-01 20:41:45,834 - 8342703936 - __init__.py-__init__:18 - WARNING: flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.


In the [validating agent output](validating_agent_output.html) example, we used a simple tool to receive and check the output.

Here we want to have access to the agent input, so we subclass the `MotleyTool` class and implement its `run` method. Inside it, we'll call an LLM critic that will assess the agent's output.

`MotleyTool` has the `exceptions_to_reflect` argument, which allows us to reflect exceptions from the tool back to the agent. This is useful for providing feedback on why the tool failed and how to fix it. By default, it reflects `InvalidOutput` exceptions, but you can pass other exceptions to it as well, even the `Exception` class for reflecting any exception.

In [5]:
class CoderOutputHandler(MotleyTool):
    def __init__(self):
        super().__init__(
            name="coder_output_handler",
            description="Output handler. ONLY RETURN THE FINAL RESULT USING THIS TOOL!",
            return_direct=True,
            exceptions_to_reflect=[InvalidOutput, ValueError],
        )  # args_schema is inferred automatically from the run method signature, but you can specify it explicitly

    def run(self, code: str, comment: str):
        agent_prompt = self.agent_input["prompt"]
        critic = LLMTool(
            name="critic",
            description="improve the agent output",
            prompt=(
                "An AI agent was given this prompt: \n{agent_prompt}\n\n"
                "Here's the code it generated: \n```\n{code}\n```\n\n"
                "And the comment it gave: \n{comment}\n\n"
                "The code and the comment must be sensible and easily understandable. "
                "Give constructive advice on how to improve them unless they are already perfect. "
                "In this case, just return a single \"OK\" without quotes."
            )
        )

        critic_message = critic.invoke({"agent_prompt": agent_prompt, "code": code, "comment": comment}).content
        print("Critic response: ", critic_message)

        if critic_message.strip().lower().startswith("ok"):
            return code, comment
        raise InvalidOutput(critic_message)  # This will be returned to the agent to make it improve its output

In case the critic is not satisfied, the `InvalidOutput` exception will be returned to the agent, as if a regular tool was called. If everything is OK, we just return the agent's output.

The ReActToolCallingMotleyAgent constructor has a `max_iterations` argument, as well as most other agents. It's the maximum number of times the agent will be allowed to call tools. If the agent doesn't provide a valid output after `max_iterations`, an exception will be raised. To avoid this, you can add an iteration counter to the critic's state.

The output schema is inferred automatically from the `run` method signature. You can also specify it directly using the `args_schema` argument.

In [6]:
crew = MotleyCrew()

coder = ReActToolCallingMotleyAgent(
    name="coder",
    tools=[PythonREPLTool(), CoderOutputHandler()],
    force_output_handler=True,
    verbose=True,
    max_iterations=15,
)

task = SimpleTask(
    crew=crew,
    name="Explain and implement the bubble sorting algorithm",
    description="Write a Python program that implements the bubble sorting algorithm "
                "and test your implementation using the REPL tool. \n"
                "Return the code and a comment explaining how it works.",
    agent=coder,
)

In [7]:
crew.run()
code, comment = task.output

2024-10-01 20:44:00,678 - motleycrew - WARNING - Lunary public key is not set, tracking will be disabled




> Entering new AgentExecutor chain...


2024-10-01 20:44:03,132 - 8342703936 - python.py-python:17 - WARNING: Python REPL can execute arbitrary code. Use with caution.



Invoking: `python_repl` with `{'command': 'def bubble_sort(arr):\n    n = len(arr)\n    for i in range(n):\n        for j in range(0, n-i-1):\n            if arr[j] > arr[j+1]:\n                arr[j], arr[j+1] = arr[j+1], arr[j]\n    return arr\n\n# Test the bubble sort implementation\nsample_list = [64, 34, 25, 12, 22, 11, 90]\nsorted_list = bubble_sort(sample_list)\nprint(sorted_list)'}`
responded: Thought: I will write a Python program that implements the bubble sort algorithm. Then, I will test the implementation using the REPL tool to ensure it works correctly. Finally, I will return the code along with a comment explaining how it works.

Let's start by writing the bubble sort implementation and testing it with a sample list.

Calling the Python REPL tool to execute the code.

[11, 12, 22, 25, 34, 64, 90]

Invoking: `coder_output_handler` with `{'code': 'def bubble_sort(arr):\n    n = len(arr)\n    for i in range(n):\n        for j in range(0, n-i-1):\n            if arr[j] > ar

See, the initial code was optimized thanks to the critic!

In [8]:
from IPython.display import display, Markdown, Code

display(Code(code, language="python"))

def bubble_sort(arr):
    n = len(arr)
    for i in range(n):
        for j in range(0, n-i-1):
            if arr[j] > arr[j+1]:
                arr[j], arr[j+1] = arr[j+1], arr[j]
    return arr

# Test the bubble sort implementation
sample_list = [64, 34, 25, 12, 22, 11, 90]
sorted_list = bubble_sort(sample_list)
print(sorted_list)

In [9]:
display(Markdown(comment))

This Python program implements the bubble sort algorithm. The `bubble_sort` function takes a list `arr` as input and sorts it in ascending order. It works by repeatedly stepping through the list, comparing adjacent elements and swapping them if they are in the wrong order. This process is repeated until the list is sorted. The nested loops ensure that each element is compared with every other element, resulting in a sorted list. The test case demonstrates the function with a sample list, and the output is the sorted list.

In [11]:
final_result = f"{code}\n\n{comment}"

# Print the final result
print(final_result)

def bubble_sort(arr):
    n = len(arr)
    for i in range(n):
        for j in range(0, n-i-1):
            if arr[j] > arr[j+1]:
                arr[j], arr[j+1] = arr[j+1], arr[j]
    return arr

# Test the bubble sort implementation
sample_list = [64, 34, 25, 12, 22, 11, 90]
sorted_list = bubble_sort(sample_list)
print(sorted_list)

This Python program implements the bubble sort algorithm. The `bubble_sort` function takes a list `arr` as input and sorts it in ascending order. It works by repeatedly stepping through the list, comparing adjacent elements and swapping them if they are in the wrong order. This process is repeated until the list is sorted. The nested loops ensure that each element is compared with every other element, resulting in a sorted list. The test case demonstrates the function with a sample list, and the output is the sorted list.
